In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
"and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [20]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review,'lxml').get_text().encode('ascii', 'ignore').decode('ascii')
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [22]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

C:\ProgramData\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


#### Creating a Min Max Vector for each review

In [73]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = []
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.vocab.keys())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec.append(model.wv[word])
    # 
    # Concatenate min and max vector for each coordinate
    featureVec1 = np.zeros((2*num_features,),dtype="float32")
    
    featureVec=np.array(featureVec)
    minv=np.minimum.reduce(featureVec)
    maxv=np.maximum.reduce(featureVec)
    featureVec1 = np.concatenate((minv,maxv))
    
    return featureVec1


def getMinMaxFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),2*num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0. :
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [74]:
# ****************************************************************
# Calculate min max feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
num_features=300
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getMinMaxFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getMinMaxFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

#### Cross validation with parameter Area under the ROC curve

In [75]:
from sklearn.model_selection import train_test_split
X_train, X_CV, y_train, y_CV = train_test_split(trainDataVecs,np.array(train['sentiment']), test_size=0.2, random_state=0)

In [76]:
import time
from sklearn.svm import LinearSVC
#from sklearn.ensemble import RandomForestClassifier
start=time.time()
#forest = RandomForestClassifier(n_estimators = 100)
#forest.fit(X_train,y_train)
svm_minmax_w2v = LinearSVC(dual=False,tol=1e-3)
svm_minmax_w2v.fit(X_train, y_train)
end = time.time()
print "svm_minmax_w2v ",end-start,"seconds"
svm_minmax_w2v.score(X_CV,y_CV)
#forest.score(X_CV, y_CV)

svm_minmax_w2v  4.24800014496 seconds


0.85419999999999996

In [77]:
y_pred=svm_minmax_w2v.predict(X_CV)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_CV, y_pred)

0.85424995070182663

### fitting on whole data

In [78]:
minmax_svm=LinearSVC(dual=False,tol=1e-3)
minmax_svm.fit(trainDataVecs,np.array(train['sentiment']))
result = minmax_svm.predict(testDataVecs)

In [79]:
# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_minmaxVectors_SVM.csv", index=False, quoting=3 )